## MLflow's Model Registry

In [33]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [34]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

#client.list_experiments()
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1685897704674, experiment_id='2', last_update_time=1685897704674, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='/home/pascal/Projects/my/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1685884178510, experiment_id='1', last_update_time=1685884178510, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [35]:
#client.create_experiment(name="nyc-taxi-experiment")

Let's check the latest versions for the experiment with id `1`...

In [36]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [37]:
for run in runs:
    print(f"run id: {run.info}, rmse: {run.data.metrics['rmse']:.4f}")

run id: <RunInfo: artifact_uri='/home/pascal/Projects/my/mlops-zoomcamp/02-experiment-tracking/mlruns/1/b924c5c96523488a80e4ec5ea2452ddd/artifacts', end_time=1685897884350, experiment_id='1', lifecycle_stage='active', run_id='b924c5c96523488a80e4ec5ea2452ddd', run_name='bouncy-skink-1000', run_uuid='b924c5c96523488a80e4ec5ea2452ddd', start_time=1685897681223, status='FINISHED', user_id='pascal'>, rmse: 6.3024
run id: <RunInfo: artifact_uri='/home/pascal/Projects/my/mlops-zoomcamp/02-experiment-tracking/mlruns/1/fcfae5b713424099942ffc998286c3e8/artifacts', end_time=1685886017526, experiment_id='1', lifecycle_stage='active', run_id='fcfae5b713424099942ffc998286c3e8', run_name='zealous-rat-78', run_uuid='fcfae5b713424099942ffc998286c3e8', start_time=1685885880768, status='FINISHED', user_id='pascal'>, rmse: 6.3024
run id: <RunInfo: artifact_uri='/home/pascal/Projects/my/mlops-zoomcamp/02-experiment-tracking/mlruns/1/e2ed788f711e4fe4a7ba960859d20019/artifacts', end_time=1685884564685, expe

### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [38]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [39]:
run_id = "b924c5c96523488a80e4ec5ea2452ddd"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor2")

Registered model 'nyc-taxi-regressor2' already exists. Creating a new version of this model...
2023/06/04 19:03:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor2, version 2
Created version '2' of model 'nyc-taxi-regressor2'.


<ModelVersion: aliases=[], creation_timestamp=1685898220198, current_stage='None', description=None, last_updated_timestamp=1685898220198, name='nyc-taxi-regressor2', run_id='b924c5c96523488a80e4ec5ea2452ddd', run_link=None, source='/home/pascal/Projects/my/mlops-zoomcamp/02-experiment-tracking/mlruns/1/b924c5c96523488a80e4ec5ea2452ddd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [42]:
model_name = "nyc-taxi-regressor2"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: None


In [43]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685898220198, current_stage='Staging', description=None, last_updated_timestamp=1685898250681, name='nyc-taxi-regressor2', run_id='b924c5c96523488a80e4ec5ea2452ddd', run_link=None, source='/home/pascal/Projects/my/mlops-zoomcamp/02-experiment-tracking/mlruns/1/b924c5c96523488a80e4ec5ea2452ddd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [44]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685898220198, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2023-06-04', last_updated_timestamp=1685898257807, name='nyc-taxi-regressor2', run_id='b924c5c96523488a80e4ec5ea2452ddd', run_link=None, source='/home/pascal/Projects/my/mlops-zoomcamp/02-experiment-tracking/mlruns/1/b924c5c96523488a80e4ec5ea2452ddd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [45]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import mlflow


def read_dataframe(filename):
    #df = pd.read_csv(filename)
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(name, stage, X_test, y_test):
    # make problems
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [46]:
df = read_dataframe("./data/green_tripdata_2021-03.parquet")

In [47]:
#client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/tmp/ipykernel_881491/3252383476.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/home/pascal/Projects/my/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [48]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [49]:
X_test = preprocess(df, dv)

In [50]:
target = "duration"
y_test = df[target].values

In [51]:
#%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

In [52]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

2023/06/04 19:04:35 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[19:04:35] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 52.9 s, sys: 315 ms, total: 53.2 s
Wall time: 30 s


{'rmse': 6.248294328516825}

In [53]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1685898220198, current_stage='Production', description='The model version 2 was transitioned to Staging on 2023-06-04', last_updated_timestamp=1685898373162, name='nyc-taxi-regressor2', run_id='b924c5c96523488a80e4ec5ea2452ddd', run_link=None, source='/home/pascal/Projects/my/mlops-zoomcamp/02-experiment-tracking/mlruns/1/b924c5c96523488a80e4ec5ea2452ddd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>